In [1]:
#OUTSTANDING ISSUES: THERE ARE DUPLICATE METABOLITES LOL 
import random
import numpy as np
import scipy as sp
import collections
import pandas as pd
from pylab import *
import seaborn as sns
import os
from matplotlib import pyplot as plt
from IPython.display import clear_output
import statsmodels.stats.multitest as multi
import sys
import warnings

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import average_precision_score

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    warnings.filterwarnings("ignore", category=UserWarning)

sns.set_style('white')
pd.options.display.float_format = '{:,.7f}'.format
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1500)

#set for reproducibility
#check current working directory

if (os.getcwd().split('\\')[-1] != 'data'):
    os.chdir('.\data')
print(os.getcwd())

C:\Users\malca\Desktop\Analysis of Biological Networks\TB-omics\tb-omics\data


In [2]:
def load_metabolomics(filename):
    # loading in TB plasma metabolomics data from tab-delimted file to pandas dataframe
    df = pd.read_csv(filename)
    df = df.rename(columns={df.columns.values[0]: 'metabolite_name'})
   
    df = df.transpose()
    df.columns = df.iloc[0, :]
    df = df.iloc[1:, :]
    df.index.name = 'sample_id'
    
    return df

In [3]:
def impute(df, thresh=0.1):
    #drop columns with proportion missing values > threshold
    null_allowed = len(df.index) * thresh
    null_columns = df.columns.values[df.isnull().sum() > null_allowed]
    df = df.drop(columns=null_columns) 
    #impute remaining nans with minimum value
    df = df.apply(lambda x: x.fillna(x.min()), axis=0)
    return df.dropna(axis=1)

In [4]:
def load_patientmetadata(filename):
    # reading in patient metadata
    p_df = pd.read_csv(filename)
    p_df.columns = p_df.columns.str.lower()
    p_df = p_df.set_index('sample_id') 
    #drop redundant columns
    p_df = p_df.drop(columns=[p_df.columns.values[0], 'id'])
    
    return p_df

def load_biochemicaldata(filename):
    b_df = pd.read_csv(filename)
    b_df.columns = b_df.columns.str.lower()
    b_df = b_df.set_index('id')
    b_df = b_df.drop(columns=[b_df.columns.values[0]])
    return b_df.reset_index()

def combine_data(p_df, m_df, b_df):
    #join with full dataset
    m_df = m_df.set_index('sample_id').join(p_df)
    
    #b_dict = dict(zip(b_df['id'], b_df['biochemical']))
    
    #m_df = m_df.rename(b_dict, axis='columns')
    return m_df.reset_index()

In [5]:
# 
def standardize_data(f_vals):
    from sklearn import preprocessing

    # applying standardization 
    scaler = preprocessing.QuantileTransformer()#StandardScaler()
    data_scaled = scaler.fit_transform(f_vals)
    
    return data_scaled

In [6]:
def make_df(f_vals, features, l_vals, labels):
    df = pd.concat([pd.DataFrame(data=l_vals, columns=labels), 
                    pd.DataFrame(data=f_vals, columns=features)], axis=1)
    return df

In [7]:
def perform_PCA(data, l_vals, labels, save=False, ncomp=10):
# computing principal components
    from sklearn import decomposition

    pcaAbs = decomposition.PCA(n_components=ncomp)
    data_PCA = pcaAbs.fit_transform(data)
    
    pc_cols = ['PC ' + str(i) for i in np.arange(1, ncomp + 1)]
    df_PCA = make_df(data_PCA, pc_cols, l_vals, labels)
    
    #Plot explained variance by number of components
    var_exp = pcaAbs.explained_variance_ratio_
    fig_ve, ax_ve = plt.subplots(1, 1)
    sns.lineplot(x=(np.arange(len(var_exp)) + 1), y=np.cumsum(var_exp), ax=ax_ve)
    plt.xlabel('PCA component number')
    plt.ylabel('Cumulative variance ratio')
    if save:
        plt.savefig('variance-exp.png', bbox_inches='tight', pad_inches=0.5)
    
    fig_pca, ax_pca = plt.subplots(1, 1)
    sns.scatterplot(x='PC 1', y='PC 2', data=df_PCA, hue='group', ax=ax_pca)
    
    return df_PCA

In [8]:
#calculate p-value on continuous data, selecting appropriate test based on normality & equal variance tests
def significanceTest(ctrl, case, alpha_normal=0.05):
    try:
        _, p_normal_ctrl = sp.stats.normaltest(ctrl, nan_policy='omit')
        _, p_normal_case = sp.stats.normaltest(case, nan_policy='omit')
    except:
        p_normal_ctrl = 1 
        p_normal_case = 1
        
    if (np.any(p_normal_ctrl < alpha_normal) and np.any(p_normal_case < alpha_normal)):
        _, p_var = sp.stats.bartlett(ctrl, case)
        _, p_diff = sp.stats.ttest_ind(ctrl, case, nan_policy='omit', equal_var=(p_var < alpha_normal))
    else:
        _, p_diff = sp.stats.ranksums(ctrl, case)
    
    return p_diff

In [9]:
def significantMetabolites(ctrl, case, features, labels, alpha_normal=0.05, alpha_diff=0.05):
    pvals = []
    logfc = []
    for metab in features:
        metab_ctrl = ctrl[metab].values 
        metab_case = case[metab].values
        if (len(metab_ctrl.shape) > 1):
            display(metab)
        p_diff = significanceTest(metab_ctrl, metab_case, alpha_normal=alpha_normal)
        pvals.append(p_diff)
        fc = np.mean(metab_case) / np.mean(metab_ctrl) #not matched in any way
        logfc.append(np.log2(fc))
    padj = multi.multipletests(pvals, alpha=alpha_diff, method='fdr_bh')
    
    significant = pd.DataFrame({'metabolite' : metabolites, 'log2fc' : logfc,
                                'p' :  pvals, 'q' : padj[1]})
    
    return significant.sort_values(by='p')

In [11]:
all_files = ['measurements_plasma_full.csv', 'measurements_serum_full.csv', 'measurements_plasmarpmi_full.csv']
all_df = []
for file in all_files:
    temp_df = impute(load_metabolomics(file))
    index = temp_df.index
    temp_df = pd.DataFrame(data=standardize_data(temp_df), columns=temp_df.columns)
    temp_df.index = index
    all_df.append(temp_df)
    
full_df   = pd.concat(all_df, sort=False).reset_index()
patient_df = load_patientmetadata('full_unblinded_metadata_with_smoking_tst.csv')
chem_df = load_biochemicaldata('biochemicals_full_list_4.csv')
full_df = combine_data(patient_df, full_df, chem_df)
#full_df.head()
full_df.to_csv('standardized_TB_metabolomes.csv')

labels = list(patient_df)
features = [x for x in list(full_df.columns) if x not in labels]
f_vals = full_df.loc[:, features].values
l_vals = full_df.loc[:, labels].values

# displaying shape and first few data entries
print('The shape of our data matrix is: ', full_df.shape)

The shape of our data matrix is:  (751, 1264)


In [12]:
full_df.head()

,sample_id,M.38002,M.34404,M.32391,M.20675,M.21049,M.34400,M.33971,M.33972,M.32497,M.39779,M.38395,M.37752,M.38768,M.38168,M.39609,M.38296,M.33228,M.35186,M.34214,M.46325,M.34397,M.45456,M.33871,M.35153,M.33822,M.44633,M.37231,M.45675,M.44563,M.47888,M.33821,M.44630,M.44560,M.34393,M.45951,M.27447,M.34419,M.36600,M.36594,M.44682,M.37419,M.47101,M.30460,M.27665,M.34395,M.34389,M.35625,M.45453,M.47087,M.21184,M.36812,M.46438,M.35628,M.45968,M.36602,M.44621,M.33230,M.21127,M.33955,M.35631,M.45970,M.35305,M.39270,M.37418,M.33961,M.42398,M.34437,M.19324,M.45966,M.46115,M.37174,M.6146,M.42374,M.43761,M.43343,M.43266,M.35256,M.34656,M.1412,M.35883,M.34258,M.37366,M.35884,M.1554,M.36847,M.36746,M.33173,M.21044,M.42489,M.37253,M.18281,M.22030,M.22036,M.35675,M.17945,M.45869,M.32506,M.35257,M.36593,M.44683,M.45095,M.35482,M.34383,M.35626,M.21232,M.46440,M.45455,M.36811,M.35819,M.35253,M.45452,M.43400,M.31675,M.41220,M.39223,M.35635,M.32197,M.39587,M.34365,M.45721,M.36803,M.34399,M.47887,M.43507,M.31787,M.36848,M.32397,M.542,M.22053,M.39600,M.1549,M.22001,M.42103,M.31943,M.27672,M.44618,M.12017,M.46165,M.46164,M.44526,M.15676,M.46547,M.46548,M.15677,M.32445,M.15749,M.3155,M.1558,M.12032,M.37475,M.44620,M.37181,M.37207,M.37209,M.37202,M.37203,M.37211,M.37210,M.36099,M.15681,M.35527,M.541,M.1669,M.32349,M.15683,M.46146,M.36098,M.1418,M.1559,M.34424,M.37184,M.37185,M.37187,M.37190,M.46172,M.37198,M.37200,M.33968,M.37372,M.31938,M.437,M.42040,M.1419,M.35136,M.1494,M.43231,M.36776,M.35692,M.35092,M.35114,M.39598,M.34390,M.38399,M.33963,M.32198,M.43258,M.554,M.555,M.2831,M.32342,M.32980,M.33084,M.1126,M.37093,M.1107,M.46552,M.40664,M.40488,M.37073,M.22132,M.33937,M.33453,M.1561,M.32827,M.31591,M.575,M.38075,M.37516,M.1118,M.1110,M.1638,M.41394,M.1640,M.512,M.42976,M.42980,M.42028,M.443,M.40068,M.22175,M.41373,M.15778,M.35838,M.12129,M.35433,M.3141,M.27414,M.35702,M.32586,M.47886,M.43807,M.2137,M.22154,M.34420,M.32412,M.569,M.39511,M.1642,M.32489,M.32492,M.15500,M.35320,M.32675,M.37112,M.22842,M.63,M.15506,M.34396,M.42592,M.38637,M.38178,M.47098,M.33970,M.1564,M.2132,M.5983,M.1712,M.1769,M.553,M.38662,M.27718,M.513,M.37077,M.37104,M.44875,M.18368,M.31453,M.35159,M.39512,M.514,M.33941,M.32425,M.36747,M.17805,M.35718,M.43424,M.36808,M.5086,M.38669,M.32415,M.44675,M.32504,M.37478,M.32417,M.32388,M.31548,M.18467,M.33587,M.33973,M.37459,M.1552,M.20699,M.33477,M.18474,M.1497,M.47112,M.47114,M.577,M.15821,M.1643,M.44876,M.42381,M.37063,M.36738,M.2730,M.34456,M.18369,M.44872,M.33422,M.33947,M.2734,M.43829,M.33420,M.18280,M.587,M.20488,M.15443,M.57,M.53,M.42994,M.44664,M.38783,M.1572,M.15122,M.27728,M.15365,M.15990,M.32338,M.32346,M.18476,M.32599,M.18477,M.42574,M.15737,M.32620,M.39379,M.34398,M.33954,M.22171,M.38151,M.33958,M.18357,M.43802,M.1573,M.41754,M.21248,M.1644,M.35678,M.32328,M.15753,M.59,M.42027,M.42084,M.41912,M.22137,M.22138,M.40266,M.33009,M.44657,M.32836,M.35322,M.43264,M.38661,M.34093,M.3127,M.17799,M.15716,M.40730,M.38116,M.27513,M.42087,M.18349,M.32405,M.1123,M.1481,M.33441,M.35437,M.1125,M.40046,M.40008,M.40067,M.42968,M.40073,M.40049,M.44656,M.34407,M.32426,M.1417,M.15140,M.527,M.39864,M.1645,M.46739,M.60,M.40010,M.40045,M.40026,M.42969,M.40030,M.37530,M.46342,M.22177,M.32455,M.1105,M.36801,M.34035,M.46223,M.40173,M.1301,M.40056,M.40020,M.42099,M.1303,M.15910,M.27723,M.15335,M.584,M.1121,M.11984,M.1302,M.40685,M.40677,M.46144,M.1584,M.19934,M.1365,M.32418,M.33952,M.31536,M.40469,M.15650,M.31609,M.35137,M.35133,M.35130,M.36752,M.35157,M.37114,M.1498,M.1585,M.45404,M.33953,M.22185,M.35665,M.37432,M.43488,M.15720,M.33943,M.27710,M.33946,M.33967,M.1587,M.1589,M.1592,M.33950,M.37496,M.37076,M.33939,M.33959,M.32390,M.1591,M.12122,M.43249,M.41820,M.2829,M.594,M.5278,M.33013,M.32462,M.37431,M.43502,M.39732,M.1356,M.42092,M.39730,M.36845,M.36754,M.33936,M.1359,M.38102,M.35160,M.46111,M.35832,M.45413,M.15736,M.1336,M.12091,M.33447,M.38165,M.37506,M.44681,M.1508,M.18254,M.36103,M.37468,M.1361,M.36817,M.32553,M.35126,M.33945,M.64,M.41374,M.41487

In [ ]:
## HOW WELL DO SAMPLE PREPS CORRELATE?
#Extract donors for which there are multiple sample types at a given timepoint
dup_df = full_df[full_df.groupby(['donor_id', 'timepoint'])['sample_type'].transform('nunique') > 1] #ends up being only paired
#For each donor at each timepoint, calculate a correlation coefficient
dup_groups = dup_df.groupby(['donor_id', 'timepoint'])

corr = []
sig = []
donors = []
times = []
sample_types = []
for (donor, time), group in dup_groups:
    sample_types.append(group['sample_type'].values)
    donors.append(donor)
    times.append(time)
    
    shared_features = group[features[1:]].dropna(axis=1).T #drop columns that are not shared
    corr_temp, sig_temp = sp.stats.pearsonr(shared_features.values[:, 0], shared_features.values[:, 1])
    corr.append(corr_temp)
    sig.append(sig_temp)

corr_df = pd.DataFrame({'donor' : donors, 'timepoint' : times, 'sample_types' : sample_types, 
                        'Pearson correlation' : corr, 'p value' : sig, 'q value' : multi.multipletests(sig, method='fdr_bh')[1]})
display(corr_df)
#Result: all sample preps correlate significantly

In [ ]:
#WHAT METABOLITES DIFFER SIGNIFICANTLY?
#Bin by time, to show when we can start detecting changes in the bulk population
full_df['time_bin'] = np.floor(np.abs(full_df['time_to_tb'] / 4)) #6 month increments
met_tp = []
for (timepoint), group in full_df.groupby(['time_bin']):
    #group = group
    ctrl = group[group['group'].str.contains('control')][features[1:]].dropna(axis=1)
    case = group[group['group'].str.contains('case')][features[1:]].dropna(axis=1)
    
    all_metabs = significantMetabolites(ctrl, case, list(ctrl), labels)
    sig_metabs = all_metabs[all_metabs['q'] <= 0.05]
    #print('Timepoint : ' + str(timepoint))
    #display(sig_metabs)
    met_tp.append(all_metabs)
    
#Result: We only see a large number of significant metabolites <6 months to TB
display(met_tp[0])


In [ ]:
display(full_df['hydroxy-CMPF*'])

In [ ]:
met_tp_site = []
near_tb = full_df[full_df['time_bin'] == 0]
for (site), group in near_tb.groupby(['site']):
    ctrl = group[group['group'].str.contains('control')][features].dropna(axis=1)
    case = group[group['group'].str.contains('case')][features].dropna(axis=1)
    
    all_metabs = significantMetabolites(ctrl, case, list(ctrl), labels)
    sig_metabs = all_metabs[all_metabs['q'] <= 0.05]
    #print('Timepoint : ' + str(timepoint))
    display(sig_metabs)
    met_tp_site.append(all_metabs)

In [ ]:
display(met_tp[0].sort_values(by='log2fc'))
display(met_tp[0].sort_values(by='q'))

In [ ]:
#WHAT METABOLITES CORRELATE WITH RISK? 
#analyzing separately by sample type (as broad as possible, color by location)
#spearman correlation with progressor status (y/n)
#pearson correlation with time to tb (metabolite-by-metabolite) 
#for a select few, show ones that go up, down, etc. relative to controls

#test on downsampled set
sig_metabs = met_tp[0][met_tp[0]['q'] <= 0.05]['metabolite'].values
metabs_to_test = met_tp[0]['metabolite'].values
#print(sig_metabs)
all_case = full_df[full_df['group'].str.contains('case')]

corr = []
sig = []
for metab in metabs_to_test:
    metab_values = all_case[metab] 
    tb_time = all_case['time_to_tb']
    
    corr_temp, sig_temp = sp.stats.spearmanr(metab_values, tb_time, nan_policy='omit')
    corr.append(corr_temp)
    sig.append(sig_temp)
    
corr_df = pd.DataFrame({'metab' : metabs_to_test, 'Pearson correlation' : corr, 
                        'p value' : sig, 'q value' : multi.multipletests(sig, method='fdr_bh')[1]})
display(corr_df.sort_values(by='q value'))
#In lumped analysis, there are some metabs that correlate with time to TB



In [ ]:
#WHAT'S THE SIGNAL TO NOISE RATIO?
#Within same individual, how does the metabolite change over time? (Means of std. dev, Pearson correlation)
#Identifying highly variable metabolites
#Pearson correlation between individuals in the same "case-control" match

all_ctrl = full_df[full_df['group'].str.contains('control')]

for (donor), group in all_ctrl.groupby('donor_id'):
    shared_features = group[features].dropna(axis=1).T #drop columns that are not shared
    
    if (shared_features.shape[1] > 1):
        corr_temp, sig_temp = sp.stats.pearsonr(shared_features.values[1:, 0], shared_features.values[1:, 1])
        display(corr_temp)

In [ ]:
#CRASHES MY NOTEBOOK WHEN I GO BY DONOR LOL CRYING
metab_var = all_ctrl.groupby('donor_id').apply(lambda x: np.mean(x) / np.std(x))
display(metab_var)

In [ ]:
#GSEA but for metabolites???? (Do we need this?)
#

In [162]:
def SVM_pred(data, features):
#Support vector machine
#classify  case vs. controls (vanilla version)
# ways to improve:
# i) Hyperparmeter serach ii) feature selection (serach for best threshold value) iii) test models
# using different time points (currently just takes all case and control data) iv) qualitative data?
# v) how much data do we need in train vs. test (+validation)

    # finding assigning labels to control (0) and case (1) subjects
    caseControlLabels = []
    for case_control in data['group']:
        if case_control == 'control':
            caseControlLabels.append(0)
        else:
            caseControlLabels.append(1)

    # extracting metabolite names -- these will be the features
    metabolites = []
    for feat in features:
        if 'M.' in feat:
            metabolites.append(feat)

    # data matrix will only consist of metabolite features (excluding other qualitative data)
    # dropping rows with nan
    fullSVM_df = data.loc[:,metabolites[0]:metabolites[-1]].dropna(axis=1)

    X_train, X_test, y_train, y_test = train_test_split(fullSVM_df, caseControlLabels, test_size=0.33, random_state=42)

    # hinge loss for linear SVM; L1 for sparsity; don't shuffle data each epoch; don't bias classes; # fairly strong bias for L1
    clf = linear_model.SGDClassifier(loss="hinge", penalty="l1", shuffle=False, class_weight="balanced" , alpha=0.001)
    featSelection = SelectFromModel(clf, threshold=0.01) # threshold for feature selection 
    model = Pipeline([
                      ('fs', featSelection), 
                      ('clf', clf), 
                    ])
    model.fit(X_train, y_train)# train model
    
    # feature weightings
    SVMFeat_df = pd.DataFrame([model.named_steps["clf"].coef_[0]], 
                                    columns=fullSVM_df.columns[model.named_steps["fs"].get_support()])
    # calculating model accurarcy
    modelAccuracy = model.score(X_test, y_test)
    
    # ROC calculation
    y_score = model.predict(X_test)
    y_score = model.decision_function(X_test)
    fpr, tpr, _ = roc_curve(y_test, y_score)
    roc_auc = auc(fpr, tpr)

    # plotting ROC
    lw = 2
    plt.figure(dpi=400, figsize=(3,3))
    plt.plot(fpr, tpr, color='darkorange',
             lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('SVM ROC')
    plt.legend(loc="lower right")
    plt.show()

    # precision recall
    plt.figure(dpi=400, figsize=(3,3))
    averagePrecision = average_precision_score(y_test, y_score)
    precision, recall, _ = precision_recall_curve(y_test, y_score)
    step_kwargs = ({'step': 'post'}
                   if 'step' in signature(plt.fill_between).parameters
                   else {})
    plt.step(recall, precision, color='b', alpha=0.2,
             where='post')
    plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title('Precision-Recall curve: AP={0:0.2f}'.format(
              averagePrecision))
